<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

# High Dimensional & Deep Learning : Autoencoders

##  What is an Autoencoder ?

<P style="text-align:center"><img src="https://blog.keras.io/img/ae/autoencoder_schema.jpg" style="float:center; display: inline" alt="schema"/></P>
<i>Architecture d'un autoencoder</i>

## Objective 

During this TP  we will build different autoencoder with Keras and Tensorflow. Here are the main objective :

* Build a autoencoder based on simple perceptron layer.
* Add regularization one layer and understand their effect.
* Build a convolutional autoencoder.
* Use a convolutional autoencoder to solve denoising problem.
* Manipulate the library in order to get and observe the result at different point of the dataflow.


The dataset used all along with TP is the MNIST dataset.

**EDIT**: 30/08/2019 Use Tensorflow V1 for this TP as Tensorflow V2 produce very unstable results.

## Library

In [ ]:
from tensorflow.keras.datasets import mnist
import tensorflow.keras.preprocessing.image as kpi
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.regularizers as kr



import numpy as np

import matplotlib.pyplot as plt

## Dataset 
As we won't apply any supervised algorithm in this TP, we do not need to load the `Y` variable.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

As seen in the previous TP, it is better to normalize the dataset before to apply algorithm on it.

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print(x_train.shape)
print(x_test.shape)

fig  = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)
x = kpi.img_to_array(x_train[0])
ax.imshow(x[:,:,0], interpolation='nearest', cmap="binary")
ax.grid(False)
plt.axis('off')
plt.show()

## Building a simple autoencoder

We will first build a very simple architecture where :

* the **encoder layer** : is a `Dense` layer composed of 32 neurons (the latent variable) with a `Relu` activation function :
$$relu(x) = max(0,x)$$,
* the **decoded layer** : is a `Dense` layer composed of  784 neurons (the input dimension) with a `Sigmoid`activation function.
$$sigmoid(x) = \frac{1}{1+\text{e}^x}$$




We first reshape the data from to be 1D.

In [ ]:
x_train_flatten = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test_flatten = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_train_flatten.shape, x_test_flatten.shape

### Write the model

**Exercice** : write the simple model described above in keras.

In [ ]:
n_latent = 32
n_input = 784

In [ ]:
# %load solutions/simple_autoencoder.py

We then learn the model. Note that the target variable are the original image.

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train_flatten, x_train_flatten, epochs=50, batch_size=256, validation_data=(x_test_flatten, x_test_flatten))

**Question** : What can you say about the choosen loss function?  How is the loss evoluting during training?

### Check outputs

We will no check how the model performs. We produce first the encoded-decoded images.

In [ ]:
decoded_test_imgs = autoencoder.predict(x_test_flatten)

The following function display both the input and the output of the autoencoder model.

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(decoded_test_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

**Question** : What can you say about this results?



### Check latent variable

The keras model that we have written above does not allow us to retrieve the latent variable. 
In order to do so, we have to re-write the model in order to get this variable late.

We first write the encoder part.

In [ ]:
encoder = km.Sequential(name="EncoderModel")
encoder.add(kl.Dense(n_latent, activation='relu', input_shape=(n_input,),name="encoder_layer"))

We then write the decoder as  another independent model

In [ ]:
decoder = km.Sequential(name="DecoderModel")
decoder.add(kl.Dense(n_input, activation='sigmoid', input_shape =(n_latent,), name = "decoded_layer" ))

We finally writhe the autoencoder model by adding the two previous model

In [ ]:
autoencoder = km.Sequential(name="EncoderDecoder")
autoencoder.add(encoder)
autoencoder.add(decoder)

The model is well composed of the association of the two previous model.

In [ ]:
autoencoder.summary()

You can acces the two sub model with the following syntax

In [ ]:
autoencoder.get_layer("EncoderModel").summary()

The model can then be learned the same way.

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train_flatten, x_train_flatten, epochs=50, batch_size=256, validation_data=(x_test_flatten, x_test_flatten))

**Question** What can you say about the loss value of the model.

We can now access and produce easily the latent variable

In [ ]:
encoded_imgs = encoder.predict(x_test_flatten)
encoded_imgs.shape

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display encoded imgs
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(8, 4))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

You can produce the decoded images by :
* Using the decoded part on the encoded images.
* Using the all architecture on the original image.

**Exercise** : Ensure that both methods produce the same results

In [ ]:
#%load solutions/decoded_images_both_method.py

## Sparse autoencoder

In the previous example the autoencoder is only constrained by the size of the hidden layer. 

In the following figure you can see the distribution of the number of latent variable set to zero for the 10.000 test images.

In [ ]:
fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(1,1,1)
ax.hist(np.sum(encoded_imgs==0,axis=1), width=0.9, bins=np.arange(-0.5,10.5,1))
ax.set_xticks(np.arange(10))
plt.show()

Another way to get a sparser encoded representation of the images is to add a *sparsity contraint* on the activity function of the hidden layer. 

Regularizers enable to avoid overfitting by adding some constraint on the weight we want to control. 

Cost function = Loss (say, binary cross entropy) + Regularization term 

Cost function = Loss + $\lambda$ $\sum w$, where in our case $\lambda = 10e-5$ and $w$ are the weight of the encoder model.

In [ ]:
l = 10e-5

sparse_encoder = km.Sequential(name="SparseEncoderModel")
sparse_encoder.add(kl.Dense(n_latent, activation='relu', input_shape=(n_input,), activity_regularizer=kr.l1(l) ,name="encoder_layer"))

sparse_decoder = km.Sequential(name="SparseDecoderModel")
sparse_decoder.add(kl.Dense(n_input, activation='sigmoid', input_shape =(n_latent,), name = "decoded_layer" ))

sparse_autoencoder = km.Sequential(name="SparseEncoderDecoder")
sparse_autoencoder.add(sparse_encoder)
sparse_autoencoder.add(sparse_decoder)


you can now train the model as previously

In [ ]:
sparse_autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
sparse_autoencoder.fit(x_train_flatten, x_train_flatten, epochs=50, batch_size=256,validation_data=(x_test_flatten, x_test_flatten))

**Question** : What can you say on the loss function compare to the previous model?

**Exercise**  : Ensure that the encoded images obtained with the sparse autoencoder are indeed sparser than the ones obtain by the first autoencoder 

In [ ]:
# %load solutions/compare_sparsity_encoded_imgs.py

**Exercise** : Compare the decoded images obtain by the first and the sparse model

In [ ]:
# %load solutions/compare_sparsity_decoded_imgs.py

## Convolutional Autoencoder

In the previous part, we have seen very simple autoencoder where both encoder and decoder part are composed of single layer. They both can be composed of more layers (deep autoencoder) and with differents types of layer.

As seen in the previous TP, convolutional layers are the best layer to use when dealing with images. 

**Exercise** : Implement a convolutional Autoencoder with the folowwing architecture: 

conv_decoder = km.Sequential(name="ConvDecoderModel")
conv_decoder.add(kl.Conv2D(8, (3, 3), activation='relu', input_shape = conv_encoder.get_output_shape_at(-1)[-3:], padding='same'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(8, (3, 3), activation='relu', padding='same'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(16, (3, 3), activation='relu'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

`Encoder`
* A 2d convolutional layer, 16 filters of size 3x3
* A 2Dmaxpooling layer with filters of size 2x2
* A 2d convolutial layer, 8 filters of size 3x3
* A 2Dmaxpooling layer with filters of size 2x2
* A 2d convolutial layer, 8 filters of size 3x3
* A 2Dmaxpooling layer with filters of size 2x2

`Decoder`
* A 2d convolutional layer, 8 filters of size 3x3
* A 2Dupsampling layer with filters of size 2x2
* A 2d convolutional layer, 8 filters of size 3x3
* A 2Dupsampling layer with filters of size 2x2
* A 2d convolutional layer, 16 filters of size 3x3
* A 2Dupsampling layer with filters of size 2x2
* A 2d convolutional layer, 1 filters of size 3x3, with SIGMOID activation


*All padding are `SAME` padding and all convolutional activation function but last are `RELU`*



In [ ]:
# %load solutions/convolutional_autoencoder.py

In [ ]:
conv_autoencoder = km.Sequential(name="ConvAutoencoderModel")
conv_autoencoder.add(conv_encoder)
conv_autoencoder.add(conv_decoder)
conv_autoencoder.summary()

In [ ]:
conv_autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
conv_autoencoder.fit(x_train_conv, x_train_conv, epochs=50, batch_size=256, validation_data=(x_test_conv, x_test_conv))

**Question** What can you say about the loss function?

In [ ]:
encoded_imgs = conv_encoder.predict(x_test_conv)
decoded_imgs = conv_autoencoder.predict(x_test_conv)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n+1)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


### Application to denoising

We now know how to build a convolutional autoencoder. 

We will now see how it can be used to solved denoising problem. 

We first create fake noisy data

In [ ]:
# Add random noise
noise_factor = 0.5
x_train_noisy = x_train_conv + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train_conv.shape) 
x_test_noisy = x_test_conv + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test_conv.shape) 

# Value greater than 1 are set to 1 and value lower than 0 are set to zero
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Let's observe the noise we create

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # noisy data
    ax = plt.subplot(2, n, i + n+1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Now let's train the same convolutional model that we built above. But let train this model with noisy data as an input and the original data as the output.

In [ ]:
conv_encoder = km.Sequential(name="ConvEncoderModel")
conv_encoder.add(kl.Conv2D(16, (3,3) , activation='relu', input_shape=(28,28,1) , padding='same' ))
conv_encoder.add(kl.MaxPooling2D((2, 2), padding='same'))
conv_encoder.add(kl.Conv2D(8, (3, 3), activation='relu', padding='same'))
conv_encoder.add(kl.MaxPooling2D((2, 2), padding='same'))
conv_encoder.add(kl.Conv2D(8, (3, 3), activation='relu', padding='same'))
conv_encoder.add(kl. MaxPooling2D((2, 2), padding='same'))

conv_decoder = km.Sequential(name="ConvDecoderModel")
conv_decoder.add(kl.Conv2D(8, (3, 3), activation='relu', input_shape = conv_encoder.get_output_shape_at(-1)[-3:], padding='same'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(8, (3, 3), activation='relu', padding='same'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(16, (3, 3), activation='relu'))
conv_decoder.add(kl.UpSampling2D((2, 2)))
conv_decoder.add(kl.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

conv_autoencoder = km.Sequential(name="ConvAutoencoderModel")
conv_autoencoder.add(conv_encoder)
conv_autoencoder.add(conv_decoder)

conv_autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
conv_autoencoder.fit(x_train_noisy, x_train_conv, epochs=10, batch_size=256, validation_data=(x_test_noisy, x_test_conv))

Now, we past the noisy test data into the trained autoencorder in order to denoise this data.

In [ ]:
x_test_denoised = conv_autoencoder.predict(x_test_noisy)

Here are the results of the denoised data

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # noisy data
    ax = plt.subplot(3, n, i + n+1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # denoised data
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(x_test_denoised[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

**Exercise** : Play with different architecture to rease loss function